In [1]:
!pip install surya-ocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/

In [1]:
!pip install pytesseract

In [3]:
!pip install qwen_vl_utils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.2/35.2 MB 35.3 MB/s eta 0:00:00


In [4]:
from PIL import Image
from surya.texify import TexifyPredictor
import re
from transformers import (
    TrOCRConfig,
    TrOCRProcessor,
    TrOCRForCausalLM,
    ViTConfig,
    ViTModel,
    VisionEncoderDecoderModel,
    AutoModelForCausalLM, AutoTokenizer
)
import os
import pandas as pd
from tqdm import tqdm
import cv2
import numpy as np
import pytesseract
import re
from pathlib import Path
from typing import Dict, Any, List, Tuple
from collections import defaultdict
import pickle
import torch
from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
from torch.utils.data import Dataset, DataLoader

# Reading Data

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [27]:
dataset_dir = "/content/data"
df = pd.read_csv("/content/one - Sheet1.csv")
mcq_df = df[df["Type"] == "MCQs"]
int_df = df[df["Type"] == "Integer"]

In [35]:
df.columns

Index(['Exam Name', 'Year', 'Subject (if any)', 'Paper Number (if any)',
       'Type', 'Image Name', 'Answer Type', 'Answers', 'Latex'],
      dtype='object')

In [37]:
mcq_data = []

for i in range(len(mcq_df)):
    row = mcq_df.iloc[i,:]

    #Only select samples with corresponding GT latex equations
    if (str(row["Latex"]).lower() == "nan"):
        continue

    ans = row["Answers"]
    latex = row["Latex"]
    img_file = dataset_dir

    for j in range(6):
        t = str(row[j])
        if (t != "-" and t.lower() != "nan"):
            img_file = os.path.join(img_file, t)
    mcq_data.append({"img_file_path":img_file, "answer": ans, "Latex": latex})


int_data = []

for i in range(len(int_df)):
    row = int_df.iloc[i,:]

    #Only select samples with corresponding GT latex equations
    if (str(row["Latex"]).lower() == "nan"):
        continue

    ans = row["Answers"]
    latex = row["Latex"]
    img_file = dataset_dir

    for j in range(6):
        t = str(row[j])
        if (t != "-" and t.lower() != "nan"):
            img_file = os.path.join(img_file, t)
    int_data.append({"img_file_path":img_file, "answer": ans, "Latex": latex})


<ipython-input-37-2501ec250b51>:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  t = str(row[j])


In [38]:
mcq_data

[{'img_file_path': '/content/data/AMC 12/2010/12A/MCQs/25.png',
  'answer': 'C',
  'Latex': 'Two quadrilaterals are considered the same if one can be obtained from the other by a rotation and a translation. How many different convex cyclic quadrilaterals are there with integer sides and perimeter equal to 32 ?\n(A) 560\n(B) 564\n(C) 568\n(D) 1498\n(E) 2255'}]

# Preprocessing Data

In [39]:
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
class MathImagePreprocessor:
    def __init__(self, target_height: int = 512, ocr_config: str = '--psm 3',
                 question_num_regex: str = r'^\(?\s*Q?\s*\d{1,2}\s*\)?\.?$'):
        self.target_height = target_height
        self.ocr_config = ocr_config
        self.question_num_regex = question_num_regex

    def _resize_image(self, image: np.ndarray) -> np.ndarray:
        height, width = image.shape[:2]
        if height > self.target_height:
            new_width = int(width * (self.target_height / height))
            return cv2.resize(image, (new_width, self.target_height), interpolation=cv2.INTER_AREA)
        return image

    def remove_question_number(self, image: np.ndarray) -> np.ndarray:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) if len(image.shape) == 3 else image
        height, width = gray.shape
        roi_width = int(width * 0.10)
        roi = gray[:, :roi_width]

        ocr_data = pytesseract.image_to_data(roi, output_type=pytesseract.Output.DICT, config='--psm 7')

        for i in range(len(ocr_data['text'])):
            text = ocr_data['text'][i].strip()
            x, y, w, h = ocr_data['left'][i], ocr_data['top'][i], ocr_data['width'][i], ocr_data['height'][i]

            if text and y < 0.15 * height and w < 0.15 * roi_width and h < 0.15 * height and x < 0.05 * roi_width:
                if re.match(r'^\(?\s*\d{1,2}\s*\)?[\.\)]?\s*$', text):
                    cut_width = min(x + w + int(0.01 * width), roi_width)
                    post_crop_strip = gray[y:y + h, cut_width:min(cut_width + int(0.05 * width), width)]
                    post_crop_text = pytesseract.image_to_string(post_crop_strip, config='--psm 7').strip()
                    if not re.match(r'^\d', post_crop_text):
                        return image[:, cut_width:]
        return image

    def detect_text_area(self, image: np.ndarray) -> np.ndarray:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) if len(image.shape) == 3 else image
        _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        non_zero_coords = cv2.findNonZero(binary)
        if non_zero_coords is None:
            return self.remove_question_number(image)
        x_coords, y_coords = non_zero_coords[:, :, 0], non_zero_coords[:, :, 1]
        min_x, max_x = np.min(x_coords), np.max(x_coords)
        min_y, max_y = np.min(y_coords), np.max(y_coords)
        padding_x = int(0.02 * (max_x - min_x))
        padding_y = int(0.02 * (max_y - min_y))
        x1, y1 = max(0, min_x - padding_x), max(0, min_y - padding_y)
        x2, y2 = min(image.shape[1], max_x + padding_x), min(image.shape[0], max_y + padding_y)
        cropped = image[y1:y2, x1:x2]
        return self.remove_question_number(cropped)

    def standardize_resolution(self, image: np.ndarray) -> np.ndarray:
        resized = self._resize_image(image)
        gray = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY) if len(resized.shape) == 3 else resized
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        enhanced = clahe.apply(gray)
        return cv2.cvtColor(enhanced, cv2.COLOR_GRAY2BGR)

    def remove_noise(self, image: np.ndarray) -> np.ndarray:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) if len(image.shape) == 3 else image
        denoised = cv2.fastNlMeansDenoising(gray, None, h=10, templateWindowSize=5, searchWindowSize=15)
        kernel = np.ones((2, 2), np.uint8)
        cleaned = cv2.morphologyEx(denoised, cv2.MORPH_OPEN, kernel)
        return cleaned

    def detect_embedded_images(self, image: np.ndarray) -> Dict[str, List[np.ndarray]]:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) if len(image.shape) == 3 else image
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        edged = cv2.Canny(blurred, 30, 150)
        contours, _ = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        image_regions = []
        height, width = image.shape[:2]

        for contour in contours:
            area = cv2.contourArea(contour)
            if area < 2500:  # Lowered area threshold to catch smaller diagrams
                continue
            x, y, w, h = cv2.boundingRect(contour)
            aspect_ratio = w / h if h > 0 else 0
            extent = float(area) / (w * h) if w * h > 0 else 0

            if 0.2 < aspect_ratio < 10 and extent < 0.95:
                roi = image[y:y + h, x:x + w]
                if h > 40 and w > 40:  # Avoid small symbols or noise
                    image_regions.append(roi)

        return {"embedded_images": image_regions}

    def segment_layout(self, image: np.ndarray) -> Dict[str, np.ndarray]:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) if len(image.shape) == 3 else image
        _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        bboxes = sorted([cv2.boundingRect(c) for c in contours], key=lambda x: x[1])
        height = image.shape[0]

        if not bboxes:
            return {"question": image, "options": None}

        def get_text_from_roi(img, bbox):
            x, y, w, h = bbox
            roi = img[y:y + h, x:x + w]
            return pytesseract.image_to_string(roi, config=self.ocr_config).strip()

        option_regex = re.compile(r'^\s*[(\[]?[A-D]\)?[\)\]\.]?\s+')
        potential_option_starts = []
        for i, bbox in enumerate(bboxes):
            text = get_text_from_roi(gray, bbox)
            if option_regex.match(text) and bbox[1] > height * 0.1:
                potential_option_starts.append((bbox[1], i))

        if potential_option_starts:
            split_y = potential_option_starts[0][0]
            return {"question": image[:split_y], "options": image[split_y:]}

        else:
            def calculate_vertical_distance(bbox1, bbox2):
                return bbox2[1] - (bbox1[1] + bbox1[3])

            max_gap = -1
            split_y = -1
            for i in range(len(bboxes) - 1):
                gap = calculate_vertical_distance(bboxes[i], bboxes[i + 1])
                if gap > max_gap and bboxes[i][1] < height * 0.6:
                    max_gap = gap
                    split_y = (bboxes[i][1] + bboxes[i][3] + bboxes[i + 1][1]) // 2

            if split_y > 0 and max_gap > 15:
                return {"question": image[:split_y], "options": image[split_y:]}
            else:
                return {"question": image[:height // 2], "options": image[height // 2:]}

    def preprocess(self, image_path: str, output_dir: str | bool) -> Dict[str, Any]:
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError(f"Could not read image at {image_path}")

        resized = self._resize_image(image)
        cropped = self.detect_text_area(resized.copy())
        standardized = self.standardize_resolution(cropped.copy())
        cleaned = self.remove_noise(standardized.copy())
        segments = self.segment_layout(cleaned.copy())
        embedded_images = self.detect_embedded_images(segments['question'])  # Focus diagram search on question part

        results = {
            'preprocessed': cleaned,
            'question': segments['question'],
            'options': segments.get('options'),
            'embedded_images': embedded_images['embedded_images']
        }

        if output_dir:
            output_dir = Path(output_dir)
            output_dir.mkdir(parents=True, exist_ok=True)
            base_name = Path(image_path).stem
            cv2.imwrite(str(output_dir / f"{base_name}_preprocessed.png"), cleaned)
            cv2.imwrite(str(output_dir / f"{base_name}_question.png"), segments['question'])
            if segments.get('options') is not None:
                cv2.imwrite(str(output_dir / f"{base_name}_options.png"), segments['options'])
            for idx, img in enumerate(embedded_images['embedded_images']):
                cv2.imwrite(str(output_dir / f"{base_name}_embedded_img_{idx}.png"), img)

        return results

In [40]:
def preprocess_data(data, target_height=512):
    preprocessor = MathImagePreprocessor(target_height=target_height)

    for d in tqdm(data, total=len(data)):
        res = preprocessor.preprocess("/content/data/25.png", False)
        d["preprocessing_results"] = res

    return data

In [41]:
print("Preprocessing MCQ Questions...")
mcq_data = preprocess_data(mcq_data)
print("Preprocessing done!")

print("Preprocessing Integer-based Questions...")
int_data = preprocess_data(int_data)
print("Preprocessing done!")

Preprocessing MCQ Questions...


100%|██████████| 1/1 [00:25<00:00, 25.46s/it]


Preprocessing done!
Preprocessing Integer-based Questions...


0it [00:00, ?it/s]

Preprocessing done!


In [42]:
mcq_data

[{'img_file_path': '/content/data/AMC 12/2010/12A/MCQs/25.png',
  'answer': 'C',
  'Latex': 'Two quadrilaterals are considered the same if one can be obtained from the other by a rotation and a translation. How many different convex cyclic quadrilaterals are there with integer sides and perimeter equal to 32 ?\n(A) 560\n(B) 564\n(C) 568\n(D) 1498\n(E) 2255',
  'preprocessing_results': {'preprocessed': array([[255, 255, 255, ..., 255, 255, 255],
          [255, 255, 255, ..., 255, 255, 255],
          [255, 255, 255, ..., 255, 255, 255],
          ...,
          [255, 255, 255, ..., 255, 255, 255],
          [255, 255, 255, ..., 255, 255, 255],
          [255, 255, 255, ..., 255, 255, 255]], dtype=uint8),
   'question': array([[255, 255, 255, ..., 255, 255, 255],
          [255, 255, 255, ..., 255, 255, 255],
          [255, 255, 255, ..., 255, 255, 255],
          ...,
          [255, 255, 255, ..., 255, 255, 255],
          [255, 255, 255, ..., 255, 255, 255],
          [255, 255, 255

In [43]:
#Save preprocessed results as pickle files
with open(os.path.join(dataset_dir, "preprocessed_data.pkl"), "wb") as f:
    pickle.dump({"mcq": mcq_data, "int": int_data} ,f)

# OCR Inference

In [44]:
def process_latex(text):
    text = text.replace("\\\\", "\\")
    text = re.sub("<math(?=\s|>)[^>]*?>", "$", text)
    text = text.replace("</math>", "$")
    return text

In [45]:
def get_latex(data, predictor, mode="mcq"):
    for d in data:
        question = Image.fromarray(d["preprocessing_results"]["preprocessed"])
        latex_gen_question = predictor([question])[0].text
#         print(latex_gen_question)
        latex_gen_question = process_latex(latex_gen_question)
        d["latex_gen_question"] = latex_gen_question

        if (mode == "mcq"):
            options = Image.fromarray(d["preprocessing_results"]["options"])
            latex_gen_options = predictor([options])[0].text
            latex_gen_options = process_latex(latex_gen_options)
            d["latex_gen_options"] = latex_gen_options

    return data

In [46]:
predictor = TexifyPredictor()
with open(os.path.join(dataset_dir, "preprocessed_data.pkl"), "rb") as f:
    preprocessed_data = pickle.load(f)

mcq_data = preprocessed_data["mcq"]
print("Generating Latex equations for MCQs...")
mcq_data = get_latex(mcq_data, predictor)
print("Done!")

int_data = preprocessed_data["int"]
print("Generating Latex equations for Integer-Based Questions...")
int_data = get_latex(int_data, predictor, mode="int")
print("Done!")

Loaded texify model s3://texify/2025_02_18 on device cuda with dtype torch.float16
Generating Latex equations for MCQs...


Texify inference: 100%|██████████| 1/1 [00:01<00:00,  1.39s/it]

Done!
Generating Latex equations for Integer-Based Questions...
Done!


# LaTeX OCR Evaluator

In [47]:
import re
from collections import defaultdict
from difflib import SequenceMatcher
from typing import List, Dict
import numpy as np
from math import isclose
import sympy as sp

class LaTeXOCREvaluator:
    def __init__(self):
        # Mathematical symbols and operators
        self.math_symbols = {
            'operators': {'+', '-', '*', '/', '=', '<', '>', '≤', '≥', '≠', '≡', '≈', '≅'},
            'relations': {'∈', '∉', '⊂', '⊆', '⊃', '⊇', '∪', '∩', '⊕', '⊗'},
            'functions': {'sin', 'cos', 'tan', 'log', 'ln', 'exp', 'lim', 'max', 'min'},
            'greek_letters': {'α', 'β', 'γ', 'δ', 'ε', 'ζ', 'η', 'θ', 'ι', 'κ', 'λ', 'μ',
                              'ν', 'ξ', 'π', 'ρ', 'σ', 'τ', 'υ', 'φ', 'χ', 'ψ', 'ω'}
        }

        # LaTeX structural elements
        self.structural_elements = {
            'environments': {'equation', 'align', 'gather', 'matrix', 'cases', 'array'},
            'commands': {'frac', 'sqrt', 'sum', 'int', 'prod', 'lim', 'binom'},
            'delimiters': {'(', ')', '[', ']', '{', '}', '|', '\\|', '⟨', '⟩'}
        }

        # Normalization rules
        self.normalization_rules = {
            r'\\left\(': '(', r'\\right\)': ')',
            r'\\left\[': '[', r'\\right\]': ']',
            r'\\left\{': '{', r'\\right\}': '}',
            r'\\big\s*': '', r'\\Big\s*': '', r'\\bigg\s*': '', r'\\Bigg\s*': '',
            r'\\,': ' ', r'\\;': ' ', r'\\:': ' ',
            r'\\text\s*\{([^}]+)\}': r'\1'
        }


    def normalize_latex(self, latex_str: str) -> str:
        """Normalize LaTeX expressions for consistent comparison"""
        latex_str = re.sub(r'%.*', '', latex_str)
        for pattern, replacement in self.normalization_rules.items():
            latex_str = re.sub(pattern, replacement, latex_str)
        latex_str = re.sub(r'\s*([=+\-*/])\s*', r' \1 ', latex_str)
        latex_str = re.sub(r'([a-zA-Z0-9])/([a-zA-Z0-9])', r'\\frac{\1}{\2}', latex_str)
        return latex_str.strip()

    def tokenize_latex(self, latex_str: str) -> List[str]:
        """Advanced tokenization of LaTeX with structure awareness"""
        tokens = []
        pos = 0
        n = len(latex_str)

        while pos < n:
            if latex_str[pos].isspace():
                pos += 1
                continue
            if latex_str[pos] == '\\':
                cmd_end = pos + 1
                while cmd_end < n and latex_str[cmd_end].isalpha():
                    cmd_end += 1
                tokens.append(latex_str[pos:cmd_end])
                pos = cmd_end
                continue
            if latex_str.startswith('\\begin{', pos):
                env_end = latex_str.find('}', pos)
                tokens.append(latex_str[pos:env_end+1])
                pos = env_end + 1
                continue
            if latex_str[pos] in {'{', '}', '[', ']', '(', ')', '|'}:
                tokens.append(latex_str[pos])
                pos += 1
                continue
            if latex_str[pos] in {'^', '_'}:
                tokens.append(latex_str[pos])
                pos += 1
                if pos < n and latex_str[pos] == '{':
                    brace_end = latex_str.find('}', pos)
                    tokens.append(latex_str[pos:brace_end+1])
                    pos = brace_end + 1
                else:
                    tokens.append(latex_str[pos])
                    pos += 1
                continue
            if latex_str[pos].isalpha():
                ident_end = pos
                while ident_end < n and latex_str[ident_end].isalnum():
                    ident_end += 1
                tokens.append(latex_str[pos:ident_end])
                pos = ident_end
                continue
            if latex_str[pos].isdigit() or latex_str[pos] == '.':
                num_end = pos
                while num_end < n and (latex_str[num_end].isdigit() or latex_str[num_end] == '.'):
                    num_end += 1
                tokens.append(latex_str[pos:num_end])
                pos = num_end
                continue
            tokens.append(latex_str[pos])
            pos += 1
        return tokens

    def compare_tokens(self, gt_tokens: List[str], pred_tokens: List[str]) -> Dict[str, float]:
        sm = SequenceMatcher(None, gt_tokens, pred_tokens)
        sequence_ratio = sm.ratio()

        gt_set = set(gt_tokens)
        pred_set = set(pred_tokens)

        tp = len(gt_set & pred_set)
        fp = len(pred_set - gt_set)
        fn = len(gt_set - pred_set)

        precision = tp / (tp + fp) if (tp + fp) > 0 else 1.0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 1.0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0

        pos_correct = sum(1 for gt, pred in zip(gt_tokens, pred_tokens) if gt == pred)
        pos_accuracy = pos_correct / max(len(gt_tokens), len(pred_tokens))

        return {
            'token_precision': precision,
            'token_recall': recall,
            'token_f1': f1,
            'sequence_similarity': sequence_ratio,
            'positional_accuracy': pos_accuracy
        }

    def detect_structural_errors(self, gt_tokens: List[str], pred_tokens: List[str]) -> Dict[str, List]:
        errors = defaultdict(list)

        def count_commands(tokens, cmd):
            return sum(1 for t in tokens if t == f'\\{cmd}')

        for cmd in self.structural_elements['commands']:
            gt_count = count_commands(gt_tokens, cmd)
            pred_count = count_commands(pred_tokens, cmd)
            if gt_count > pred_count:
                errors['missing_commands'].append(f'\\{cmd}')
            elif gt_count < pred_count:
                errors['extra_commands'].append(f'\\{cmd}')

        delimiter_pairs = {'(': ')', '[': ']', '{': '}'}
        for open_delim, close_delim in delimiter_pairs.items():
            gt_open = gt_tokens.count(open_delim)
            gt_close = gt_tokens.count(close_delim)
            pred_open = pred_tokens.count(open_delim)
            pred_close = pred_tokens.count(close_delim)
            if gt_open != gt_close:
                errors['gt_delimiter_mismatch'].append(f'{open_delim}{close_delim}')
            if pred_open != pred_close:
                errors['pred_delimiter_mismatch'].append(f'{open_delim}{close_delim}')

        for i, token in enumerate(gt_tokens):
            if token in {'^', '_'}:
                if i >= len(pred_tokens) or pred_tokens[i] != token:
                    errors['superscript_subscript_errors'].append(
                        f"Expected '{token}' at position {i}, found '{pred_tokens[i] if i < len(pred_tokens) else 'EOF'}'"
                    )

        return dict(errors)

    def semantic_equivalence(self, gt_expr: str, pred_expr: str) -> float:
        try:
            gt_parsed = sp.sympify(gt_expr, evaluate=False)
            pred_parsed = sp.sympify(pred_expr, evaluate=False)

            if gt_parsed == pred_parsed:
                return 1.0

            variables = list(gt_parsed.free_symbols.union(pred_parsed.free_symbols))
            test_values = np.random.uniform(0.1, 10.0, size=(5, len(variables)))

            matches = 0
            for values in test_values:
                subs = dict(zip(variables, values))
                try:
                    gt_val = float(gt_parsed.evalf(subs=subs))
                    pred_val = float(pred_parsed.evalf(subs=subs))
                    if isclose(gt_val, pred_val, rel_tol=1e-5):
                        matches += 1
                except Exception:
                    continue

            return matches / len(test_values) if len(test_values) > 0 else 0.0

        except Exception:
            return SequenceMatcher(None, gt_expr, pred_expr).ratio()

    def evaluate_expression(self, gt_expr: str, pred_expr: str) -> Dict:
        gt_norm = self.normalize_latex(gt_expr)
        pred_norm = self.normalize_latex(pred_expr)

        gt_tokens = self.tokenize_latex(gt_norm)
        pred_tokens = self.tokenize_latex(pred_norm)

        token_metrics = self.compare_tokens(gt_tokens, pred_tokens)
        structural_errors = self.detect_structural_errors(gt_tokens, pred_tokens)
        semantic_score = self.semantic_equivalence(gt_norm, pred_norm)

        composite_score = (
            0.4 * token_metrics['token_f1'] +
            0.3 * token_metrics['sequence_similarity'] +
            0.2 * token_metrics['positional_accuracy'] +
            0.1 * semantic_score
        )

        return {
            # 'gt': gt_expr,
            # 'pred': pred_expr,
            # 'normalized_gt': gt_norm,
            # 'normalized_pred': pred_norm,
            'token_metrics': token_metrics,
            'structural_errors': structural_errors,
            'semantic_equivalence': semantic_score,
            'composite_score': composite_score
        }

In [49]:
evaluator = LaTeXOCREvaluator()

# Add OCR evaluation scores for MCQs
for d in mcq_data:
    gt_options = d.get("Latex", "")  # Ground truth
    pred_options = d.get("latex_gen_options", "")  # OCR-generated
    result = evaluator.evaluate_expression(gt_options, pred_options)
    d["ocr_metrics_options"] = result

for d in mcq_data[:5]:
    print("GT:", d["Latex"])
    print("Predicted:", d["latex_gen_options"])
    print("Metrics:", d["ocr_metrics_options"])
    print("Score:", d["ocr_metrics_options"]["composite_score"])
    print("----")


GT: Two quadrilaterals are considered the same if one can be obtained from the other by a rotation and a translation. How many different convex cyclic quadrilaterals are there with integer sides and perimeter equal to 32 ?
(A) 560
(B) 564
(C) 568
(D) 1498
(E) 2255
Predicted: $\begin{array}{cccc} \text{(A)} \ 560 & \text{(B)} \ 564 & \text{(C)} \ 568 & \text{(D)} \ 1498 & \text{(E)} \ 2255 \end{array}$
Metrics: {'token_metrics': {'token_precision': 0.5714285714285714, 'token_recall': 0.26666666666666666, 'token_f1': 0.36363636363636365, 'sequence_similarity': 0.4, 'positional_accuracy': 0.0}, 'structural_errors': {}, 'semantic_equivalence': 0.23796033994334279, 'composite_score': 0.2892505794488798}
Score: 0.2892505794488798
----


In [50]:
mcq_data

[{'img_file_path': '/content/data/AMC 12/2010/12A/MCQs/25.png',
  'answer': 'C',
  'Latex': 'Two quadrilaterals are considered the same if one can be obtained from the other by a rotation and a translation. How many different convex cyclic quadrilaterals are there with integer sides and perimeter equal to 32 ?\n(A) 560\n(B) 564\n(C) 568\n(D) 1498\n(E) 2255',
  'preprocessing_results': {'preprocessed': array([[255, 255, 255, ..., 255, 255, 255],
          [255, 255, 255, ..., 255, 255, 255],
          [255, 255, 255, ..., 255, 255, 255],
          ...,
          [255, 255, 255, ..., 255, 255, 255],
          [255, 255, 255, ..., 255, 255, 255],
          [255, 255, 255, ..., 255, 255, 255]], dtype=uint8),
   'question': array([[255, 255, 255, ..., 255, 255, 255],
          [255, 255, 255, ..., 255, 255, 255],
          [255, 255, 255, ..., 255, 255, 255],
          ...,
          [255, 255, 255, ..., 255, 255, 255],
          [255, 255, 255, ..., 255, 255, 255],
          [255, 255, 255

In [51]:
d

{'img_file_path': '/content/data/AMC 12/2010/12A/MCQs/25.png',
 'answer': 'C',
 'Latex': 'Two quadrilaterals are considered the same if one can be obtained from the other by a rotation and a translation. How many different convex cyclic quadrilaterals are there with integer sides and perimeter equal to 32 ?\n(A) 560\n(B) 564\n(C) 568\n(D) 1498\n(E) 2255',
 'preprocessing_results': {'preprocessed': array([[255, 255, 255, ..., 255, 255, 255],
         [255, 255, 255, ..., 255, 255, 255],
         [255, 255, 255, ..., 255, 255, 255],
         ...,
         [255, 255, 255, ..., 255, 255, 255],
         [255, 255, 255, ..., 255, 255, 255],
         [255, 255, 255, ..., 255, 255, 255]], dtype=uint8),
  'question': array([[255, 255, 255, ..., 255, 255, 255],
         [255, 255, 255, ..., 255, 255, 255],
         [255, 255, 255, ..., 255, 255, 255],
         ...,
         [255, 255, 255, ..., 255, 255, 255],
         [255, 255, 255, ..., 255, 255, 255],
         [255, 255, 255, ..., 255, 255, 

In [52]:
Image.fromarray(int_data[0]["preprocessing_results"]["options"])

IndexError: list index out of range

In [56]:
with open(os.path.join("", "ocr_out.pkl"), "wb") as f:
    pickle.dump({"mcq": mcq_data, "int": int_data}, f)

# VLM
- Qwen 2.5 Math 1.5B Instruct
- Qwen 2.5 Math 7B Instruct

In [57]:
#Hyperparameters
batch_size = 2

#Load data
with open(os.path.join("", "ocr_out.pkl"), "rb") as f:
    ocr_out = pickle.load(f)

In [58]:
class MathDataset(Dataset):
    def __init__(self, ocr_out, mode="int"):
        super(MathDataset, self).__init__()
        self.data = ocr_out[mode] #(img_file_path, GT answer, Latex returned by OCR model)

    def __getitem__(self, idx):
        return {
            "img_file_path": self.data[idx][0],
            "answer": self.data[idx][1],
            "latex_ocr": self.data[idx][2]
        }

    def __len__(self):
        return len(self.data)

In [59]:
with open(os.path.join("", "ocr_out.pkl"), "rb") as f:
    out = pickle.load(f)

dataset_mcq = MathDataset(out, mode="mcq")
dataset_int = MathDataset(out, mode="int")

# dataloader_mcq = DataLoader(dataset_mcq, batch_size=batch_size, num_workers=0, shuffle=False)
# dataloader_int = DataLoader(dataset_int, batch_size=batch_size, num_workers=0, shuffle=False)

In [60]:
dataset_int

### Qwen 2.5 Math 1.5B Instruct

In [61]:
model_name = "Qwen/Qwen2.5-Math-1.5B-Instruct"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/656 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.32k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [66]:
ocr_out["mcq"]

[{'img_file_path': '/content/data/AMC 12/2010/12A/MCQs/25.png',
  'answer': 'C',
  'Latex': 'Two quadrilaterals are considered the same if one can be obtained from the other by a rotation and a translation. How many different convex cyclic quadrilaterals are there with integer sides and perimeter equal to 32 ?\n(A) 560\n(B) 564\n(C) 568\n(D) 1498\n(E) 2255',
  'preprocessing_results': {'preprocessed': array([[255, 255, 255, ..., 255, 255, 255],
          [255, 255, 255, ..., 255, 255, 255],
          [255, 255, 255, ..., 255, 255, 255],
          ...,
          [255, 255, 255, ..., 255, 255, 255],
          [255, 255, 255, ..., 255, 255, 255],
          [255, 255, 255, ..., 255, 255, 255]], dtype=uint8),
   'question': array([[255, 255, 255, ..., 255, 255, 255],
          [255, 255, 255, ..., 255, 255, 255],
          [255, 255, 255, ..., 255, 255, 255],
          ...,
          [255, 255, 255, ..., 255, 255, 255],
          [255, 255, 255, ..., 255, 255, 255],
          [255, 255, 255

In [72]:

# # CoT
# messages = [
#     {"role": "system", "content": "Please reason step by step, and put your final answer within \\boxed{}."},
#     {"role": "user", "content": prompt}
# ]



for q_type in ["int", "mcq"]:
    data = ocr_out[q_type]
    responses = []
    for idx, sample in tqdm(enumerate(data), total=len(data)):
        question = sample["latex_gen_question"]
        options = sample["latex_gen_options"] if "latex_gen_options" in sample else ""

        # TIR
        if (len(options) > 0):
            messages = [
                {"role": "system", "content": "Only prvide the final answer, and put your final answer within \\boxed{}"},
                {"role": "user", "content": f"Question: {question}"},
                {"role": "user", "content": f"Options: {options}"}
            ]
        else:
            messages = [
                {"role": "system", "content": "Only prvide the final answer, and put your final answer within \\boxed{}"},
                {"role": "user", "content": f"Question: {question}"},
            ]
        print(messages)

        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        model_inputs = tokenizer([text], return_tensors="pt").to(device)

        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=512
        )
        generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]

        response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        responses.append(response)
        # sample["gen_answer": response]
        # break

    # ocr_out[q_type][]

#     if (idx >= 1):
#         break

0it [00:00, ?it/s]
  0%|          | 0/1 [00:00<?, ?it/s]

[{'role': 'system', 'content': 'Only prvide the final answer, and put your final answer within \\boxed{}'}, {'role': 'user', 'content': 'Question: $\\begin{array}{l} \\text{Two quadrilaterals are considered the same if one can be obtained from the other by a rotation and a translation. How many different curves are } \\mathbf{A} \\text{ and } \\mathbf{B} \\text{ are } \\mathbf{A} \\text{ and } \\mathbf{B} \\text{ is } \\mathbf{A}^2 \\text{ and } \\mathbf{B}^3. \\ \\text{c cyclic quadrilateral are there with integer sides and perimeter equal to 32?} \\ \\text{(A) 560 \\qquad \\text{(B) 564} \\qquad \\text{(C) 568 \\qquad \\text{(D) 1498} \\qquad \\text{(E) 2255} \\end{array}$'}, {'role': 'user', 'content': 'Options: $\\begin{array}{cccc} \\text{(A)} \\ 560 & \\text{(B)} \\ 564 & \\text{(C)} \\ 568 & \\text{(D)} \\ 1498 & \\text{(E)} \\ 2255 \\end{array}$'}]


100%|██████████| 1/1 [00:29<00:00, 29.31s/it]


In [73]:
print(responses)

["To solve the problem, we need to determine the number of different cyclic quadrilaterals with integer sides and a perimeter of 32. A cyclic quadrilateral is one where the sum of the lengths of opposite sides is equal. Let the sides of the quadrilateral be \\(a, b, c,\\) and \\(d\\). The perimeter condition gives us:\n\n\\[a + b + c + d = 32.\\]\n\nFor the quadrilateral to be cyclic, it must satisfy the condition that the sum of the lengths of opposite sides is equal. This means:\n\n\\[a + c = b + d.\\]\n\nLet's denote \\(a + c = b + d = s\\). Then we have:\n\n\\[2s = 32 \\implies s = 16.\\]\n\nSo, we need to find all sets of integers \\((a, c)\\) and \\((b, d)\\) such that \\(a + c = 16\\) and \\(b + d = 16\\), and \\(a, b, c, d\\) are all positive integers. The possible pairs \\((a, c)\\) and \\((b, d)\\) are:\n\n\\[\n\\begin{aligned}\n&(1, 15), (2, 14), (3, 13), (4, 12), (5, 11), (6, 10), (7, 9), (8, 8), (9, 7), (10, 6), (11, 5), (12, 4), (13, 3), (14, 2), (15, 1).\n\\end{aligned}\

### Qwen 2.5 VL 3B Instruct

In [74]:
# default: Load the model on the available device(s)
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-3B-Instruct", torch_dtype="auto", device_map="auto"
)

# We recommend enabling flash_attention_2 for better acceleration and memory saving, especially in multi-image and video scenarios.
# model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
#     "Qwen/Qwen2.5-VL-3B-Instruct",
#     torch_dtype=torch.bfloat16,
#     attn_implementation="flash_attention_2",
#     device_map="auto",
# )

# default processer
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct")

# The default range for the number of visual tokens per image in the model is 4-16384.
# You can set min_pixels and max_pixels according to your needs, such as a token range of 256-1280, to balance performance and cost.
min_pixels = 256*28*28
max_pixels = 1280*28*28
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct", min_pixels=min_pixels, max_pixels=max_pixels)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/216 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json:   0%|          | 0.00/5.70k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [81]:
sample = next(iter(ocr_out["mcq"]))
question = sample["latex_gen_question"]+sample["latex_gen_options"]
# img = [Image.open(sample["img_file_path"][i]) for i in range(batch_size)]
system_prompt = "Please integrate natural language reasoning with programs to solve the problem above, and put your final answer within \\boxed{}"

messages = []
for i in range(batch_size):
    message = [
        {"role": "system", "content": system_prompt},
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "image": Image.open("/content/data/25.png"),
                },
                {"type": "text", "text": question[i]},
            ],
        }
    ]
    messages.append(message)

# Preparation for inference
# text = processor.apply_chat_template(
#     messages, tokenize=False, add_generation_prompt=True
# )
texts = [
    processor.apply_chat_template(msg, tokenize=False, add_generation_prompt=True)
    for msg in messages
]
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=texts,
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    padding_style="left",
    return_tensors="pt",
)
inputs = inputs.to("cuda")

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=512)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)

Keyword argument `padding_style` is not a valid argument for this processor and will be ignored.


In [ ]:
print(output_text[0])

To solve this problem, we need to find all positive integers less than 1000 that satisfy two conditions:
1. The sum of the digits of the number is divisible by 7.
2. The number itself is divisible by 3.

Let's break down the solution step-by-step:

### Step 1: Identify Numbers Less Than 1000
First, we list all numbers from 1 to 999 (since 1000 is not included).

### Step 2: Check Divisibility by 3
A number is divisible by 3 if the sum of its digits is divisible by 3. However, since we are also checking divisibility by 7, we need to ensure that the sum of the digits is divisible by both 3 and 7.

### Step 3: Sum of Digits Divisible by 7
We need to find numbers whose digit sums are multiples of 7. This means the possible sums of the digits are 7, 14, 21, etc., up to a maximum of 63 (since the maximum sum for a three-digit number is \(9 + 9 + 9 = 27\), but we need it to be divisible by 7).

### Step 4: Check for Divisibility by 3
For each valid sum of digits, we need to check if the numbe

# OCR Training (Ignore for now)
Requires parallel textual corpus

In [ ]:
# TrOCR is a decoder model and should be used within a VisionEncoderDecoderModel
# init vision2text model with random weights
encoder = ViTModel(ViTConfig())
decoder = TrOCRForCausalLM(TrOCRConfig())
model = VisionEncoderDecoderModel(encoder=encoder, decoder=decoder)

# If you want to start from the pretrained model, load the checkpoint with `VisionEncoderDecoderModel`
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-handwritten")

preprocessor_config.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [ ]:
class OCRDataset(Dataset):
    def __init__(self, processor, img_file_names):
        super(OCRDataset, self).__init__()
        self.img_file_names = img_file_names
        self.processor = processor

    def __getitem__(self, idx):
        img = Image.open(img_file_names[idx]).convert("RGB")
        pixel_values = processor(img, return_tensors="pt").pixel_values
        return pixel_values

    def __len__(self):
        return len(self.img_file_names)

In [ ]:
img_files_train = []
img_files_val = []
img_files_test = []

train_dataset = OCRDataset(processor, img_files_train)
val_dataset = OCRDataset(processor, img_files_val)
test_dataset = OCRDataset(processor, img_files_test)

In [ ]:


# load image from the IAM dataset
url = "https://fki.tic.heia-fr.ch/static/img/a01-122-02.jpg"
image = Image.open(requests.get(url, stream=True).raw).convert("RGB")
pixel_values = processor(image, return_tensors="pt").pixel_values
text = "industry, ' Mr. Brown commented icily. ' Let us have a"

# training
model.config.decoder_start_token_id = processor.tokenizer.eos_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id
model.config.vocab_size = model.config.decoder.vocab_size

labels = processor.tokenizer(text, return_tensors="pt").input_ids
outputs = model(pixel_values, labels=labels)
loss = outputs.loss
round(loss.item(), 2)

# inference
generated_ids = model.generate(pixel_values)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
generated_text